In [101]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [102]:
np.random.seed(0)
data = pd.read_csv("Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("Data\\cle\\Rand_valid.csv", sep=',')

In [103]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X
X2_val = X_val

In [104]:
cut = 0
y1 = (Y <= cut) * 1
y2 = (Y > cut) * (-1) * (-Y)
Y2 = np.stack((y1, y2), axis=-1)

y1_val = (Y_val <= cut) * 1
y2_val = (Y_val > cut) * (-1) * (-Y_val)
Y2_val = np.stack((y1_val, y2_val), axis=-1)

In [105]:
from fit9_nn import fit, comb_error

In [128]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(16, 24, bias = True)
        self.fc12 = nn.Linear(24, 1, bias = True)
       
        
        self.fc21 = nn.Linear(16, 24, bias=True)
        self.fc22 = nn.Linear(24, 5, bias=True)
        self.fc23 = nn.Linear(5, 1, bias=True)
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = torch.sigmoid(self.fc12(x1))
        
        x2 = self.fc21(x)
        x2 = torch.tanh(self.fc22(x2))
        x2 = torch.exp(self.fc23(x2))       
        
        x3 = torch.cat((x1, x2), 1)
        return x3

In [129]:
n_epochs = 2000
lr = 0.0001
verbose = 1
n_batches = 5
batch_to_avg = 2
ep_to_check = 10
clipping = 0.20
sig2 = 2 * 950000
PATH = 'Data\\checkpoints\\mytraining2.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [130]:
def m_error(output, target):
    return comb_error(output, target, sig2)

In [131]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [132]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.data.uniform_(-y, y)

In [133]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = 'Data\\checkpoints\\mytraining2.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error, 
                n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)

Epoch 1: Training Loss: 0.9880306720733643, Validation Loss: 289.8818359375
Epoch 2: Training Loss: 0.9866426587104797, Validation Loss: 289.87701416015625
Epoch 3: Training Loss: 0.9852407455444336, Validation Loss: 289.87200927734375
Epoch 4: Training Loss: 0.9839021921157837, Validation Loss: 289.8668518066406
Epoch 5: Training Loss: 0.9825271248817444, Validation Loss: 289.8614807128906
Epoch 6: Training Loss: 0.9811864733695984, Validation Loss: 289.8559875488281
Epoch 7: Training Loss: 0.9799271941184997, Validation Loss: 289.85028076171875
Epoch 8: Training Loss: 0.9785973191261291, Validation Loss: 289.84429931640625
Epoch 9: Training Loss: 0.9773199319839477, Validation Loss: 289.8379821777344
Epoch 10: Training Loss: 0.9760497212409973, Validation Loss: 289.831298828125
Epoch 11: Training Loss: 0.974757981300354, Validation Loss: 289.8241882324219
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 0.9735066652297973, Validation Loss: 289.8166198730469
Epoch 13: Training 

Epoch 101: Training Loss: 0.9051483154296875, Validation Loss: 286.44964599609375
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.9047075629234314, Validation Loss: 286.43572998046875
Epoch 103: Training Loss: 0.904278552532196, Validation Loss: 286.42193603515625
Epoch 104: Training Loss: 0.9038426160812378, Validation Loss: 286.408447265625
Epoch 105: Training Loss: 0.9033948540687561, Validation Loss: 286.3951721191406
Epoch 106: Training Loss: 0.9029739379882813, Validation Loss: 286.38189697265625
Epoch 107: Training Loss: 0.9025609970092774, Validation Loss: 286.36883544921875
Epoch 108: Training Loss: 0.9021338939666748, Validation Loss: 286.35595703125
Epoch 109: Training Loss: 0.9017210960388183, Validation Loss: 286.34326171875
Epoch 110: Training Loss: 0.9012850284576416, Validation Loss: 286.3305969238281
Epoch 111: Training Loss: 0.9008925914764404, Validation Loss: 286.3181457519531
New Checkpoint Saved into PATH
Epoch 112: Training Loss: 0.900475823879242, Val

Epoch 199: Training Loss: 0.8712141394615174, Validation Loss: 285.3905029296875
Epoch 200: Training Loss: 0.8709582567214966, Validation Loss: 285.3801574707031
Epoch 201: Training Loss: 0.8706886053085328, Validation Loss: 285.3697814941406
New Checkpoint Saved into PATH
Epoch 202: Training Loss: 0.8704448342323303, Validation Loss: 285.3591003417969
Epoch 203: Training Loss: 0.8701921820640564, Validation Loss: 285.3483581542969
Epoch 204: Training Loss: 0.8699460625648499, Validation Loss: 285.337646484375
Epoch 205: Training Loss: 0.8696955919265748, Validation Loss: 285.3270568847656
Epoch 206: Training Loss: 0.8694442272186279, Validation Loss: 285.31634521484375
Epoch 207: Training Loss: 0.8692014694213868, Validation Loss: 285.3057861328125
Epoch 208: Training Loss: 0.8689514398574829, Validation Loss: 285.29510498046875
Epoch 209: Training Loss: 0.8687128782272339, Validation Loss: 285.2845764160156
Epoch 210: Training Loss: 0.8684823393821717, Validation Loss: 285.2741394042

Epoch 297: Training Loss: 0.856119430065155, Validation Loss: 284.27801513671875
Epoch 298: Training Loss: 0.8560579895973206, Validation Loss: 284.2662353515625
Epoch 299: Training Loss: 0.8559832811355591, Validation Loss: 284.2545471191406
Epoch 300: Training Loss: 0.8559198975563049, Validation Loss: 284.2425231933594
Epoch 301: Training Loss: 0.8558524012565613, Validation Loss: 284.23101806640625
New Checkpoint Saved into PATH
Epoch 302: Training Loss: 0.8557826042175293, Validation Loss: 284.21942138671875
Epoch 303: Training Loss: 0.8557165265083313, Validation Loss: 284.2076721191406
Epoch 304: Training Loss: 0.8556577801704407, Validation Loss: 284.1955871582031
Epoch 305: Training Loss: 0.8555935144424438, Validation Loss: 284.1837158203125
Epoch 306: Training Loss: 0.8555303096771241, Validation Loss: 284.17181396484375
Epoch 307: Training Loss: 0.8554765224456787, Validation Loss: 284.15948486328125
Epoch 308: Training Loss: 0.8554119944572449, Validation Loss: 284.1475830

Epoch 395: Training Loss: 0.8516304850578308, Validation Loss: 283.2071228027344
Epoch 396: Training Loss: 0.851593804359436, Validation Loss: 283.1955261230469
Epoch 397: Training Loss: 0.8515629768371582, Validation Loss: 283.1848449707031
Epoch 398: Training Loss: 0.8515262722969055, Validation Loss: 283.1738586425781
Epoch 399: Training Loss: 0.8514977097511292, Validation Loss: 283.1637878417969
Epoch 400: Training Loss: 0.8514639019966126, Validation Loss: 283.15380859375
Epoch 401: Training Loss: 0.8514300346374511, Validation Loss: 283.1431884765625
New Checkpoint Saved into PATH
Epoch 402: Training Loss: 0.8513978362083435, Validation Loss: 283.1322937011719
Epoch 403: Training Loss: 0.851364815235138, Validation Loss: 283.1211242675781
Epoch 404: Training Loss: 0.8513309001922608, Validation Loss: 283.1105651855469
Epoch 405: Training Loss: 0.8512972831726074, Validation Loss: 283.0994873046875
Epoch 406: Training Loss: 0.8512672781944275, Validation Loss: 283.0876770019531
E

...

Epoch 1578: Training Loss: 0.8190450549125672, Validation Loss: 275.6805419921875
Epoch 1579: Training Loss: 0.8189996957778931, Validation Loss: 275.6912841796875
Epoch 1580: Training Loss: 0.8189732789993286, Validation Loss: 275.69171142578125
Epoch 1581: Training Loss: 0.8189178705215454, Validation Loss: 275.6950988769531
Epoch 1582: Training Loss: 0.8188804507255554, Validation Loss: 275.7103271484375
Epoch 1583: Training Loss: 0.8188501954078674, Validation Loss: 275.7470703125
Epoch 1584: Training Loss: 0.8188139200210571, Validation Loss: 275.78399658203125
Epoch 1585: Training Loss: 0.818784761428833, Validation Loss: 275.82763671875
Epoch 1586: Training Loss: 0.8187464952468873, Validation Loss: 275.8527526855469
Epoch 1587: Training Loss: 0.8187082409858704, Validation Loss: 275.88775634765625
Epoch 1588: Training Loss: 0.8186690092086792, Validation Loss: 275.92559814453125
Epoch 1589: Training Loss: 0.818636131286621, Validation Loss: 275.9709777832031
Epoch 1590: Trainin

Epoch 1680: Training Loss: 0.8153230667114257, Validation Loss: 280.2678527832031
Epoch 1681: Training Loss: 0.8152844667434692, Validation Loss: 280.318359375
Epoch 1682: Training Loss: 0.8152448534965515, Validation Loss: 280.3681640625
Epoch 1683: Training Loss: 0.8152084350585938, Validation Loss: 280.43341064453125
Epoch 1684: Training Loss: 0.8151761054992676, Validation Loss: 280.5113220214844
Epoch 1685: Training Loss: 0.8151360273361206, Validation Loss: 280.576904296875
Epoch 1686: Training Loss: 0.8150997757911682, Validation Loss: 280.6394348144531
Epoch 1687: Training Loss: 0.815065574645996, Validation Loss: 280.68463134765625
Epoch 1688: Training Loss: 0.8150300264358521, Validation Loss: 280.7568359375
Epoch 1689: Training Loss: 0.8149941086769104, Validation Loss: 280.81494140625
Epoch 1690: Training Loss: 0.8149517416954041, Validation Loss: 280.85797119140625
Epoch 1691: Training Loss: 0.8149143934249878, Validation Loss: 280.8984069824219
Epoch 1692: Training Loss: 

Epoch 1782: Training Loss: 0.8116396546363831, Validation Loss: 286.70220947265625
Epoch 1783: Training Loss: 0.8116116523742676, Validation Loss: 286.7910461425781
Epoch 1784: Training Loss: 0.8115792155265809, Validation Loss: 286.8340759277344
Epoch 1785: Training Loss: 0.8115377187728882, Validation Loss: 286.89410400390625
Epoch 1786: Training Loss: 0.8115065693855286, Validation Loss: 286.9727478027344
Epoch 1787: Training Loss: 0.8114731669425964, Validation Loss: 287.03851318359375
Epoch 1788: Training Loss: 0.8114365339279175, Validation Loss: 287.1194763183594
Epoch 1789: Training Loss: 0.811398983001709, Validation Loss: 287.2010192871094
Epoch 1790: Training Loss: 0.8113666415214539, Validation Loss: 287.3012390136719
Epoch 1791: Training Loss: 0.8113248586654663, Validation Loss: 287.4084777832031
Epoch 1792: Training Loss: 0.8112964153289794, Validation Loss: 287.5396728515625
Epoch 1793: Training Loss: 0.811244547367096, Validation Loss: 287.65875244140625
Epoch 1794: Tr

Epoch 1884: Training Loss: 0.8082422733306884, Validation Loss: 295.13690185546875
Epoch 1885: Training Loss: 0.8082082152366639, Validation Loss: 295.2511901855469
Epoch 1886: Training Loss: 0.8081725716590882, Validation Loss: 295.36236572265625
Epoch 1887: Training Loss: 0.8081412434577941, Validation Loss: 295.443359375
Epoch 1888: Training Loss: 0.8081183314323426, Validation Loss: 295.5581359863281
Epoch 1889: Training Loss: 0.8080836534500122, Validation Loss: 295.61676025390625
Epoch 1890: Training Loss: 0.8080581307411194, Validation Loss: 295.6418151855469
Epoch 1891: Training Loss: 0.808014178276062, Validation Loss: 295.6962890625
Epoch 1892: Training Loss: 0.8079909086227417, Validation Loss: 295.7551574707031
Epoch 1893: Training Loss: 0.8079609513282776, Validation Loss: 295.8415832519531
Epoch 1894: Training Loss: 0.8079212784767151, Validation Loss: 295.92706298828125
Epoch 1895: Training Loss: 0.8078930974006653, Validation Loss: 296.00146484375
Epoch 1896: Training L

Epoch 1986: Training Loss: 0.8051452755928039, Validation Loss: 300.63720703125
Epoch 1987: Training Loss: 0.8051185488700867, Validation Loss: 300.7578125
Epoch 1988: Training Loss: 0.8050897359848023, Validation Loss: 300.8210144042969
Epoch 1989: Training Loss: 0.8050663352012635, Validation Loss: 300.9289855957031
Epoch 1990: Training Loss: 0.8050417184829712, Validation Loss: 300.9898681640625
Epoch 1991: Training Loss: 0.8050139307975769, Validation Loss: 301.0258483886719
Epoch 1992: Training Loss: 0.804981791973114, Validation Loss: 301.1254577636719
Epoch 1993: Training Loss: 0.8049560546875, Validation Loss: 301.216064453125
Epoch 1994: Training Loss: 0.8049304723739624, Validation Loss: 301.2846984863281
Epoch 1995: Training Loss: 0.8049002647399902, Validation Loss: 301.4313049316406
Epoch 1996: Training Loss: 0.8048718333244324, Validation Loss: 301.51654052734375
Epoch 1997: Training Loss: 0.8048501014709473, Validation Loss: 301.6337890625
Epoch 1998: Training Loss: 0.80